In [1]:
from parseMD import *

In [2]:
from dynpy_params import ParseDynamics as PD

In [4]:
u,vel = PARSE_MD(PD.traj_dir+'01/',PD)

In [5]:
u.atom.head()

,x,y,z,frame,symbol
atom,,,,,
581806,28.214288,6.335806,4.614078,30000,I
581807,4.427381,4.917798,23.619779,30000,H
581808,3.262754,2.427441,22.963625,30000,H
581809,7.035619,8.913404,-1.763328,30000,H
581810,4.233696,9.350260,-0.736154,30000,H


In [6]:
u.atom['label'] = u.atom.get_atom_labels()
u.frame.loc[:,'time'] = u.frame.index*PD.timestep

u.compute_atom_two(vector=True,bond_extra=0.9)

In [7]:
u.compute_molecule()

In [8]:
u.atom.head()

,x,y,z,frame,symbol,label,molecule
atom,,,,,,,
581806,28.214288,6.335806,4.614078,30000,I,0,11
581807,4.427381,4.917798,23.619779,30000,H,1,12
581808,3.262754,2.427441,22.963625,30000,H,2,12
581809,7.035619,8.913404,-1.763328,30000,H,3,13
581810,4.233696,9.350260,-0.736154,30000,H,4,13


In [9]:
u.molecule.classify(('H(2)O(1)','water',True))

C:\Users\Adam\anaconda3\lib\site-packages\exatomic\core\molecule.py:73: UserWarning: Unclassified molecules remaining...
  warnings.warn("Unclassified molecules remaining...")


In [10]:
u.molecule

,H,I,O,mass,classification
molecule,,,,,
11,0,1,0,126.904473,NaN
12,2,0,1,18.015280,water
13,2,0,1,18.015280,water
14,2,0,1,18.015280,water
15,2,0,1,18.015280,water
...,...,...,...,...,...
713,2,0,1,18.015280,water
714,2,0,1,18.015280,water
715,2,0,1,18.015280,water


In [22]:
u.atom['classification'] = u.atom.molecule.map(u.molecule.classification)
u.atom = u.atom[u.atom['classification'] == 'water']

In [137]:
labels = pd.DataFrame(u.atom.groupby('molecule').apply(lambda x: tuple(x['label'])),columns=['mol_atom_labels'])

In [154]:
mol_labels = labels.groupby('mol_atom_labels').nunique().reset_index().reset_index().rename(columns={'index':'molecule_label'}).set_index('mol_atom_labels')

In [155]:
mol_labels

,molecule_label
mol_atom_labels,
(),0
"(1, 2, 130)",1
"(3, 4, 131)",2
"(5, 6, 132)",3
"(7, 96, 133)",4
...,...
"(119, 120, 189)",59
"(121, 122, 190)",60
"(123, 124, 191)",61


In [156]:
labels

,mol_atom_labels
11,()
12,"(1, 2, 130)"
13,"(3, 4, 131)"
14,"(5, 6, 132)"
15,"(7, 96, 133)"
...,...
713,"(119, 120, 189)"
714,"(121, 122, 190)"
715,"(123, 124, 191)"
716,"(125, 126, 192)"


In [158]:
labels['molecule_label'] = labels.mol_atom_labels.map(mol_labels.molecule_label)

In [159]:
labels

,mol_atom_labels,molecule_label
11,(),0
12,"(1, 2, 130)",1
13,"(3, 4, 131)",2
14,"(5, 6, 132)",3
15,"(7, 96, 133)",4
...,...,...
713,"(119, 120, 189)",59
714,"(121, 122, 190)",60
715,"(123, 124, 191)",61
716,"(125, 126, 192)",62


In [160]:
u.atom['molecule_label'] = u.atom.molecule.map(labels.molecule_label)

In [162]:
u.atom.tail()

,x,y,z,frame,symbol,label,molecule,classification,mol-atom-labels,molecule_label
atom,,,,,,,,,,
583935,4.623802,4.680830,16.364438,30100,O,189,713,water,"(119, 120, 189)",59
583936,20.039441,17.580835,7.547549,30100,O,190,714,water,"(121, 122, 190)",60
583937,19.922546,3.849359,16.247290,30100,O,191,715,water,"(123, 124, 191)",61
583938,13.776932,18.619916,0.597200,30100,O,192,716,water,"(125, 126, 192)",62
583939,10.034659,26.764515,3.820099,30100,O,193,717,water,"(127, 128, 193)",63


In [197]:
sets = u.atom.groupby('frame').apply(lambda x: set(list(x['molecule_label']))).values.tolist()

In [201]:
contiguos_molecules = set.intersection(*sets)

In [202]:
u.atom = u.atom[u.atom['molecule_label'].isin(contiguos_molecules)]

In [203]:
u.atom

,x,y,z,frame,symbol,label,molecule,classification,mol-atom-labels,molecule_label
atom,,,,,,,,,,
581807,4.427381,4.917798,23.619779,30000,H,1,12,water,"(1, 2, 130)",1
581808,3.262754,2.427441,22.963625,30000,H,2,12,water,"(1, 2, 130)",1
581809,7.035619,8.913404,-1.763328,30000,H,3,13,water,"(3, 4, 131)",2
581810,4.233696,9.350260,-0.736154,30000,H,4,13,water,"(3, 4, 131)",2
581811,15.435521,5.141563,26.333745,30000,H,5,14,water,"(5, 6, 132)",3
...,...,...,...,...,...,...,...,...,...,...
583935,4.623802,4.680830,16.364438,30100,O,189,713,water,"(119, 120, 189)",59
583936,20.039441,17.580835,7.547549,30100,O,190,714,water,"(121, 122, 190)",60
583937,19.922546,3.849359,16.247290,30100,O,191,715,water,"(123, 124, 191)",61


In [4]:
acfs = pd.read_csv("./example-data/QE/water/01/Jacfs_all.csv")

In [5]:
acfs

,Unnamed: 0,$J_{x}$,$J_{y}$,$J_{z}$,frame,molecule_label,molecule
0,0,NaN,NaN,NaN,30000,1,0
1,0,NaN,NaN,NaN,30010,1,61
2,0,NaN,NaN,NaN,30020,1,123
3,0,NaN,NaN,NaN,30030,1,185
4,0,NaN,NaN,NaN,30040,1,247
...,...,...,...,...,...,...,...
397,0,NaN,NaN,NaN,31960,2,11918
398,0,NaN,NaN,NaN,31970,2,11978
399,0,NaN,NaN,NaN,31980,2,12038
400,0,NaN,NaN,NaN,31990,2,12098


In [6]:
from SRparse import *

In [9]:
import dynpy_params as params

In [12]:
SpinRotation(params)

List of trajectories not provided. All will be parsed...
The following directories will be parsed as trajectories:
./example-data/QE/water/01
compute_atom_two                        --- 13.611825704574585 seconds ---
compute_molecule                        --- 6.895731449127197 seconds ---


C:\Users\Adam\anaconda3\lib\site-packages\exatomic\core\molecule.py:73: UserWarning: Unclassified molecules remaining...
  warnings.warn("Unclassified molecules remaining...")


sort,slice,and add masses to dfs        --- 2.562037467956543 seconds ---
map columns to atom_two                 --- 0.21096181869506836 seconds ---
group dataframes by molecule            --- 0.0020003318786621094 seconds ---
parallel compute angular vel,momentum   --- 11.974836826324463 seconds ---
parallel compute acfs                   --- 44.58631110191345 seconds ---
1/T1     =     nan Hz
Total SR Run Time     --- 44.74931001663208 seconds ---


In [63]:
vel = pd.read_csv("./vel.csv")
#atom = pd.read_csv("./atom.csv")
vel

,Unnamed: 0,symbol,x,y,z,frame
0,0,O,0.343552D+01,0.164631D+01,-0.895100D+00,0
1,1,H,-0.694970D+01,0.207495D+02,-0.678917D-01,0
2,2,H,-0.531751D+02,0.185476D+02,0.242310D+02,0
3,3,O,0.285894D+01,0.646562D+01,0.326353D+01,0
4,4,H,0.198045D+02,0.911515D+01,0.229146D+01,0
...,...,...,...,...,...,...
6595,6595,H,0.169173D+02,0.716370D+01,0.290746D+02,10
6596,6596,H,-0.140890D+02,0.135674D+02,0.297809D+01,10
6597,6597,O,0.294320D+01,0.383972D+00,-0.397763D+01,10
6598,6598,H,-0.274846D+01,-0.822809D+01,0.125772D+02,10


In [17]:
from SRrax import correlate

In [19]:
J.groupby('molecule_label').apply(correlate,columns_in=['x','y','z'],columns_out=['$J_{x}$','$J_{y}$','$J_{z}$'],pass_columns=['frame','molecule_label','molecule'])

C:\Users\Adam\AppData\Local\Temp\ipykernel_8480\2107077702.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  J.groupby('molecule_label').apply(correlate,columns_in=['x','y','z'],columns_out=['$J_{x}$','$J_{y}$','$J_{z}$'],pass_columns=['frame','molecule_label','molecule'])


,$J_{x}$,$J_{y}$,$J_{z}$,frame,molecule_label,molecule
0,NaN,NaN,NaN,30000,1,0
1,NaN,NaN,NaN,30000,2,1
2,NaN,NaN,NaN,30010,1,61
3,NaN,NaN,NaN,30010,2,62
4,NaN,NaN,NaN,30020,1,123
...,...,...,...,...,...,...
397,NaN,NaN,NaN,31980,2,12038
398,NaN,NaN,NaN,31990,1,12097
399,NaN,NaN,NaN,31990,2,12098
400,NaN,NaN,NaN,32000,1,12157


In [57]:
vel

,atom,symbol,x,y,z,frame,label
0,600,O,-42.098948,111.295417,165.247620,1,0
1,601,H,-39.527389,104.223114,162.651325,1,1
2,602,H,-40.000198,104.374293,170.747944,1,2
3,603,O,-111.382685,-41.736073,-65.839010,1,3
4,604,H,-104.401174,-54.857704,-72.939102,1,4
...,...,...,...,...,...,...,...
5995,6595,H,-7.351907,75.578030,53.531096,10,595
5996,6596,H,-17.181113,71.479392,48.307069,10,596
5997,6597,O,-1.558505,-181.136709,83.911971,10,597
5998,6598,H,-0.541936,-187.608872,80.445976,10,598


In [58]:
atom

,atom,symbol,x,y,z,frame,label


In [46]:
vel['molecule'] = vel.index.map(atom['molecule'])

In [55]:
vel

,atom,x,y,z,frame,symbol,label,molecule
0,576000,-9.715625,0.915395,0.384324,30010,O,0,0.0
1,576001,-0.598988,-8.296343,9.754943,30010,H,1,0.0
2,576002,-2.933537,-3.503136,3.121120,30010,H,2,0.0
3,576003,-11.223066,-1.955803,-0.894785,30010,O,3,1.0
4,576004,9.731307,-4.580917,-15.700483,30010,H,4,1.0
...,...,...,...,...,...,...,...,...
38395,614395,0.940950,0.958078,-2.122938,32000,H,187,NaN
38396,614396,-29.337230,-3.261558,-7.062871,32000,H,188,NaN
38397,614397,70.524246,11.781382,3.033590,32000,O,189,NaN
38398,614398,6.479746,-18.157718,-38.550089,32000,H,190,NaN
